---
## Importing packages

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from collections import deque
import glob

---
## Camera calibration

In [2]:
# Step 1: define the calibration function
def get_camera_calibration(calib_file):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,9,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    imgpoints = []
    objpoints = []
    # Make a list of calibration images
    images = glob.glob(calib_file)
    # Step through the list and search for chessboard corners
    img_calib_show = []
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)        
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            img_calib_show.append(img) # for visualization    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
            
    return mtx, dist

In [3]:
# Step 2: do the calibration using the images for calibration.
global mtx
global dist
mtx, dist = get_camera_calibration('./camera_cal/calibration*.jpg')

---
## Pipeline(video)

In [4]:
# Step 1: define functions for the detection
#====================camera_distortion_correction===================================
def camera_distortion_correction(img, mtx, dist):      
    # Undistort using mtx and dist
    undist = cv2.undistort(img, mtx, dist, None, mtx)        
    return undist
#===================thresholded_binary====================================
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1    
    return binary_output

def abs_sobel_thresh(img,sobel_kernel=3, orient='x',thresh=(0,255)):
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    sobelx = cv2.Sobel(gray, cv2.CV_64F,1,0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F,0,1, ksize=sobel_kernel)
    if orient=='x':
        sobel = sobelx
    elif orient=='y':
        sobel = sobely
    else:
        sobel = np.sqrt(np.square(sobelx)+np.square(sobely))
    # 3) Take the absolute value of the derivative or gradient
    sobel_abs = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    sobel_scaled = np.uint8(255*sobel_abs/np.max(sobel_abs))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
    binary_output = np.zeros_like(sobel_scaled)
    binary_output[(sobel_scaled >= thresh[0]) & (sobel_scaled <= thresh[1])] = 1   
    return binary_output

def hls_select(img, s_thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > s_thresh[0]) & (s_channel <= s_thresh[1])] = 1    
    return binary_output

def thresholded_binary(img,ksize = 5):        
    gradx = abs_sobel_thresh(img, orient='x', sobel_kernel=ksize, thresh=(40, 100))
    grady = abs_sobel_thresh(img, orient='y', sobel_kernel=ksize, thresh=(0, 255))
    gradxy = abs_sobel_thresh(img, orient='xy', sobel_kernel=ksize, thresh=(30, 100))
    dir_binary = dir_threshold(img, sobel_kernel=15, thresh=(0.8, 1.3))
    s_binary = hls_select(img, s_thresh=(180, 255))
    
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((gradxy == 1) & (dir_binary == 1)) | (s_binary == 1)] = 1     
    return combined
#================birds-eye view transfer=======================================
def birds_eye_transfer(img,flag=1):
    '''
    img: binary image
    '''
    img_size = (img.shape[1], img.shape[0])
    offset = 90
    src = np.float32([[559,477],[280,680],[1035,680],[728,477]])
    dst = np.float32([[280+offset,330],[280+offset,680],[1035-offset,680],[1035-offset,330]])
    if flag==1:
        M = cv2.getPerspectiveTransform(src, dst)
    elif flag==2:
        M = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(img, M, img_size)    
    return warped

def region_of_interest(img):
    '''
    img: binary image
    '''
    shape = img.shape 
    vertices = np.array([[(90,shape[0]),(shape[1]-90,shape[0]),(shape[1]-570,420),(570,420)]],dtype=np.int32)
    mask = np.zeros_like(img)   
    ignore_mask_color = 255        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
#================get line inds & pixels=======================================
def get_line_inds(img,linex_base,nwindows=9,margin=70,minpix = 50):    
    window_height = np.int(img.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    linex_current = linex_base
    # Create empty lists to receive left and right lane pixel indices
    line_inds = []    
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_x_low = linex_current - margin
        win_x_high = linex_current + margin
        if linex_base<img.shape[1]/2:
            good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_x_low) &  (nonzerox < win_x_high)).nonzero()[0]
        else:
            good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_x_low) &  (nonzerox < win_x_high)).nonzero()[0]
        # Append these indices to the lists
        line_inds.append(good_inds)       
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_inds) > minpix:
            linex_current = np.int(np.mean(nonzerox[good_inds]))                   
    # Concatenate the arrays of indices
    line_inds = np.concatenate(line_inds)              
    return line_inds

def pixels_on_lane(line,lane_inds):    
    line_fit = line.avg_fit_coeffs #######avg       
    ploty = line.fit_yvals
    line_fitx = line_fit[0]*ploty**2 + line_fit[1]*ploty + line_fit[2]        
    return line_fitx
#================line detection quality check=======================================
## done by Line.line_check()
#================show boundaries=======================================
def show_boundaries(img_orig,img_perspective_trans,left_fitx,right_fitx,ploty):
    warp_zero = np.zeros((img_perspective_trans.shape[0],img_perspective_trans.shape[1])).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
        
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    newwarp = birds_eye_transfer(color_warp,flag=2) 
    # Combine the result with the original image
    result = cv2.addWeighted(img_orig, 1.0, newwarp, 0.6, 0)
    
    return result
#================visual display=======================================
def visual_display(img,video,curverad,offset):
    if video:
        # Add text information on the video frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        text_pos = 'Pos of the car: '+str(np.round(offset*100))+ ' cm'+'(+ right)'        
        if curverad >= 10000:
            radius = 'Inf'
        else:
            radius = str(curverad)
        text_rad = 'Radius: '+radius+ ' m'
        cv2.putText(img,text_pos,(10,25), font, 1,(255,255,255),2)
        cv2.putText(img,text_rad,(10,75), font, 1,(255,255,255),2)
        return img  

In [5]:
# Step2: define the pipeline
#================PipeLine=======================================
def get_lane_pipeline(img):
    
    imshape = img.shape    
    # Apply a distortion correction to raw images.
    img_undist = camera_distortion_correction(img, mtx, dist)
    # Create a thresholded binary image.
    img_thresholded_binary = thresholded_binary(img_undist,ksize = 5)    
    img_thresholded_binary = region_of_interest(img_thresholded_binary)    
    # Apply a perspective transform to rectify binary image ("birds-eye view").
    img_perspective_trans = birds_eye_transfer(img_thresholded_binary,flag=1)
    # Detect lane pixels and fit to find the lane boundary.
    yval = max(left.fit_yvals)
    if left.detected:
        left_base = left.line_pos[0]
    elif left.n_buffered>0 :
        left_base= left.avg_fit_coeffs[0]*yval**2 + left.avg_fit_coeffs[1]*yval + left.avg_fit_coeffs[2]
    else:
        left_base = imshape[1]*1/4
        
    if right.detected:
        right_base = right.line_pos[1]
    elif right.n_buffered>0  :
        right_base= right.avg_fit_coeffs[0]*yval**2 + right.avg_fit_coeffs[1]*yval + right.avg_fit_coeffs[2]
    else:
        right_base = imshape[1]*3/4
    
    left_lane_inds = get_line_inds(img_perspective_trans,left_base)
    right_lane_inds = get_line_inds(img_perspective_trans,right_base)        
    left.detected,left.n_buffered = left.update(img_perspective_trans,left_lane_inds,left_base)
    right.detected,right.n_buffered = right.update(img_perspective_trans,right_lane_inds,right_base)
    # Determine the curvature of the lane and vehicle position with respect to center.    
    curvarad = np.round(left.radius_of_curvature+right.radius_of_curvature)/2    
    offset = (np.abs(left.line_offset)-np.abs(right.line_offset))/2
    # Warp the detected lane boundaries back onto the original image.
    left_fitx = pixels_on_lane(left,left_lane_inds)
    right_fitx = pixels_on_lane(right,right_lane_inds)
    masked_img = show_boundaries(img_undist,img_perspective_trans,left_fitx,right_fitx,left.fit_yvals)
    # Output visual display.
    final_img = visual_display(masked_img,1,curvarad,offset)
    
    return final_img


---
## Define a class to receive the characteristics of each line detection

In [6]:
class Line:
    def __init__(self,n=5):
        # length of queue to store data
        self.n = n
        #number of fits in buffer
        self.n_buffered = 0
        # was the line detected in the last iteration?
        self.detected = False          
        # x values of the last n fits of the line
        self.recent_xfitted = deque([],maxlen=n)
        # xvals of the most recent fit
        self.current_fit_xvals = [np.array([False])]          
        #average x values of the fitted line over the last n iterations
        self.avgx = None       
        # x diff
        self.x_diff = 0
        
        # fit coeffs of the last n fits
        self.recent_fit_coeffs = deque([],maxlen=n)
        #polynomial coefficients for the most recent fit
        self.current_fit_coeffs = [np.array([False])]    
        #polynomial coefficients averaged over the last n iterations
        self.avg_fit_coeffs = None  
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float')         
              
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        #y values for line fit
        self.fit_yvals = np.linspace(0, 100, num=101)*7.2  # always the same y-range as image
        #radius of curvature of the line in some units
        self.radius_of_curvature = 10000 
        self.recent_curvatures = deque([],maxlen=n)
        self.avg_curvature = None
        # origin (pixels) of fitted line at the bottom of the image
        self.line_pos = np.array([320,960])         
        self.current_fit_base = None        
        #distance in meters of vehicle center from the line
        self.line_offset = None         
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float')     
    
    def set_current_fit_xvals(self):
        '''
        Get x coordinates of fitted points in the current frame 
        '''
        yvals = self.fit_yvals
        yval = max(yvals)
        self.current_fit_xvals = self.current_fit_coeffs[0]*yvals**2 + self.current_fit_coeffs[1]*yvals + self.current_fit_coeffs[2]
        self.current_fit_base = self.current_fit_coeffs[0]*yval**2 + self.current_fit_coeffs[1]*yval + self.current_fit_coeffs[2]
    
    def add_data(self):
        '''
        Add new fitting information of current frames into the buffer
        '''
        self.recent_xfitted.appendleft(self.current_fit_xvals)
        self.recent_fit_coeffs.appendleft(self.current_fit_coeffs)
        ##
        self.recent_curvatures.appendleft(self.radius_of_curvature)
        assert len(self.recent_xfitted)==len(self.recent_fit_coeffs)
        self.n_buffered = len(self.recent_xfitted)
        
    def pop_data(self):   
        '''
        Delete the fitting information of frames that fails to detect lines
        '''
        if self.n_buffered>0:
            self.recent_xfitted.pop()
            self.recent_fit_coeffs.pop()
            ##
            self.recent_curvatures.pop()
            assert len(self.recent_xfitted)==len(self.recent_fit_coeffs)
            self.n_buffered = len(self.recent_xfitted)
        
        return self.n_buffered
    
    def set_avgx(self):
        '''
        Get the average x coordinates of fitted points 
        '''
        fits = self.recent_xfitted
        if len(fits)>0:
            avg=0
            for fit in fits:
                avg +=np.array(fit)
            avg = avg / len(fits)
            self.avgx = avg
    
    def set_avgcoeffs(self):
        '''
        Average the fit coefficents to improve robustness
        '''
        coeffs = self.recent_fit_coeffs
        if len(coeffs)>0:
            avg=0
            for coeff in coeffs:
                avg +=np.array(coeff)
            avg = avg / len(coeffs)
            self.avg_fit_coeffs = avg
            
    def set_avg_curvature(self):
        '''
        Average the fit coefficents to improve robustness
        '''
        curs = self.recent_curvatures
        if len(curs)>0:
            avg=0
            for cur in curs:
                avg +=np.array(cur)
            avg = avg / len(curs)
            self.avg_curvature = avg
    
    def set_allxy(self,img,lane_inds):
        '''
        Get the coordinates of points in search windows
        '''
        nonzero = img.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        self.allx = nonzerox[lane_inds]
        self.ally = nonzeroy[lane_inds] 
        
    def set_current_fit_coeffs(self):
        '''
        Get the fit coefficents of the line in current image
        '''
        self.current_fit_coeffs = np.polyfit(self.ally, self.allx, 2)        
    
    def set_radius_of_curvature(self):
        '''
        Get the curvature of the line in real world
        '''
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        y_eval = max(self.fit_yvals)        
        if self.avg_fit_coeffs is not None:
            avg_fitx = self.avg_fit_coeffs[0]*self.fit_yvals**2 + self.avg_fit_coeffs[1]*self.fit_yvals + self.avg_fit_coeffs[2]                
            avg_fit_cr = np.polyfit(self.fit_yvals*ym_per_pix, avg_fitx*xm_per_pix, 2)
            self.radius_of_curvature = ((1 + (2*avg_fit_cr[0]*y_eval*ym_per_pix + avg_fit_cr[1])**2)**1.5) / np.absolute(2*avg_fit_cr[0])
        else:
            cur_fitx = self.current_fit_coeffs[0]*self.fit_yvals**2 + self.current_fit_coeffs[1]*self.fit_yvals + self.current_fit_coeffs[2]                
            cur_fit_cr = np.polyfit(self.fit_yvals*ym_per_pix, cur_fitx*xm_per_pix, 2)
            self.radius_of_curvature = ((1 + (2*cur_fit_cr[0]*y_eval*ym_per_pix + cur_fit_cr[1])**2)**1.5) / np.absolute(2*cur_fit_cr[0])
        return self.radius_of_curvature           
    def set_line_pos(self,img):
        '''
        Use histogram to get the two base point of the lines
        '''
        histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        self.line_pos = [leftx_base,rightx_base]        
        
    def set_line_offset(self,line_base):
        '''
        To get the distance(m) between car center and the line's down points
        '''
        if line_base<640:
            self.line_offset = (self.line_pos[0] - 640)*3.7/700.0 
        else:
            self.line_offset = (self.line_pos[1] - 640)*3.7/700.0 
   
    def set_x_diff(self,line_base):
        '''
        To get the different between current x coordinate and average x coordinate of the line's down point 
        '''
        if(self.n_buffered > 0):
            yval = max(self.fit_yvals)
            avg_x = self.avg_fit_coeffs[0]*yval**2 + self.avg_fit_coeffs[1]*yval + self.avg_fit_coeffs[2]
            current_x = line_base
            self.x_diff = current_x-avg_x      
           
    def line_check(self):
        '''
        Use line_offset & x_diff to check whether the detected result counts.
        '''
        flag = True
        maxdist = 2.3  # distance in meters from the lane
        maxxdiff = 90        
        max_base = 675        
        max_base_diff = 95        
        #max_cur_diff = 200
        
        base_diff = [abs(self.line_pos[0]-self.current_fit_base),abs(self.line_pos[1]-self.current_fit_base)]
                
        if(abs(self.line_offset) > maxdist ):
            print('lane offsets too much',self.line_offset)
            flag  = False  
        
        if(min(base_diff) > max_base_diff ):
            print('base differs too much',min(base_diff))
            flag  = False          
       
        if(self.n_buffered > 0):
            if self.x_diff>maxxdiff:
                print('x differs too much',self.x_diff)
                flag=False 
            '''
            cur_diff = abs(self.avg_curvature-self.radius_of_curvature)
            if(cur_diff> max_cur_diff ):
                print('Curvature differs too much',cur_diff)
                flag  = False
            '''        
        if (abs(self.line_pos[0]-self.line_pos[1])>max_base):
            print('lane width too large',abs(self.line_pos[0]-self.line_pos[1]))
            flag  = False
        
        return flag    
        
    def update(self,img,lane_inds,line_base):
        '''
        Fit the current image and determine if the detected result is good enough to put into the buffer.
        '''        
        self.set_allxy(img,lane_inds)
        self.set_current_fit_coeffs()
        self.set_current_fit_xvals()
        self.set_radius_of_curvature()
        self.set_line_pos(img)
        self.set_line_offset(line_base)
        self.set_x_diff(line_base)       
        if self.line_check():
            self.detected=True
            self.add_data()
            self.set_avgx()
            self.set_avgcoeffs()  
            self.set_avg_curvature()
        else:
            self.detected=False            
            self.pop_data()            
            if self.n_buffered>0:
                self.set_avgx()
                self.set_avgcoeffs()  
                self.set_avg_curvature()
                    
        return self.detected,self.n_buffered     

----
## Detect lane lines in the project video

In [7]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [8]:
global left
global right

left = Line(9)
right = Line(9)
output = 'test_videos_output/processed_project_video1.mp4'
clip = VideoFileClip("project_video.mp4")
out_clip = clip.fl_image(get_lane_pipeline)
%time out_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video test_videos_output/processed_project_video1.mp4
[MoviePy] Writing video test_videos_output/processed_project_video1.mp4


 44%|███████████████████████████████████▏                                           | 561/1261 [02:54<03:25,  3.40it/s]

lane width too large 678
lane width too large 678


 45%|███████████████████████████████████▏                                           | 562/1261 [02:54<03:29,  3.33it/s]

lane width too large 682
lane width too large 682


...

x differs too much 91.591679614


 84%|█████████████████████████████████████████████████████████████████▋            | 1061/1261 [05:29<01:02,  3.20it/s]

x differs too much 111.6419149


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [06:34<00:00,  3.25it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/processed_project_video1.mp4 

Wall time: 6min 35s
